In [0]:
WITH transacoes AS (
    SELECT  
        v.idVendedor AS vendedor, 
        v.descUF AS UfVendedor, 
        c.idClienteUnico AS cliente, 
        c.descUF AS UfCliente, 
        date_format(p.dtPedido, 'yyyy-MM') AS mesAno
    FROM silver.olist.pedido p
    JOIN silver.olist.item_pedido i ON i.idPedido = p.idPedido
    JOIN silver.olist.cliente c ON c.idCliente = p.idCliente
    JOIN silver.olist.vendedor v ON v.idVendedor = i.idVendedor
    WHERE p.descSituacao <> 'canceled'
      AND p.dtPedido BETWEEN '2018-01-01' AND '2018-07-02' -- Últimos 6 meses
),
acum AS (
    SELECT 
        t.vendedor, 
        t.cliente, 
        t.UfCliente, 
        t.mesAno, 
        COUNT(*) AS comprasNoMes,
        SUM(COUNT(*)) OVER (PARTITION BY t.vendedor, t.cliente ORDER BY t.mesAno ROWS BETWEEN 6 PRECEDING AND 1 PRECEDING) AS comprasUltimos6Meses
    FROM transacoes t
    GROUP BY t.vendedor, t.cliente, t.UfCliente, t.mesAno
),
classificacao AS (
    SELECT 
        a.vendedor, 
        a.cliente, 
        a.UfCliente, 
        a.mesAno, 
        a.comprasNoMes,
        a.comprasUltimos6Meses,
        CASE 
            WHEN a.comprasNoMes > 0 AND a.comprasUltimos6Meses IS NULL THEN 'Cliente Novo'
            WHEN a.comprasNoMes = 1 AND a.comprasUltimos6Meses = 1 THEN 'Cliente Pontual'
            WHEN a.comprasNoMes > 0 AND a.comprasUltimos6Meses >= 2 THEN 'Cliente Recorrente'
            ELSE 'Outro'
        END AS categoriaCliente
    FROM acum a
)
SELECT 
    '2018-06' AS referencia, 
    c.vendedor, 
    c.cliente, 
    c.UfCliente, 
    c.mesAno, 
    c.comprasNoMes, 
    c.comprasUltimos6Meses, 
    c.categoriaCliente
FROM classificacao c
WHERE c.mesAno = '2018-06'
ORDER BY  c.mesAno, c.vendedor